In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [6]:
from autorocks.data.loader.all_models_result_aggregator import create_all_models_comparison_dataset
from autorocks.global_flags import DEVICE
import gpytorch.means as means
from botorch.utils.transforms import normalize

from autorocks.data.dataset import BOSystemDataset
from autorocks.data.debugger import OfflineDebugger
from autorocks.dir_struct import LocalResultDir
from autorocks.envs.synthetic.funcs.ackley import Ackley6DParametersSpace
from autorocks.optimizer.bograph.dag_dao.model_nodes.node_singlegp import SingleTaskGPModelNode
import torch


data = create_all_models_comparison_dataset(
    exp_dir=LocalResultDir / "synthetic/target/Akcley/6_params/100_iter", save_results=False
)

historic_data_df = data.combine_sys_params_metric()
sys_param = Ackley6DParametersSpace()
dataset = BOSystemDataset(
    historic_data_df=historic_data_df,
    parameters_name=["x1", "x2", "x3", "x4", "x5", "x6"],
    objectives_name=["target"],
    transform=lambda X: normalize(X, bounds=torch.tensor(sys_param.bounds().T)),
    target_transform=lambda X: -X,
)

subset_dataset = (
    data.filter_for_specific_models({"BoTorch_SingleTaskGP"})
    .filter_col_for_values("iteration", {1})
    .combine_sys_params_metric()
)

specific_model_dataset = BOSystemDataset(
    historic_data_df=subset_dataset,
    parameters_name=["x1", "x2", "x3", "x4", "x5", "x6"],
    objectives_name=["target"],
    transform=lambda X: normalize(X, bounds=torch.tensor(sys_param.bounds().T)),
    target_transform=lambda X: -X,
)
debugger = OfflineDebugger(
    observed_dataset=dataset,
    model_callable=lambda train_x, train_y: SingleTaskGPModelNode(
        train_x=train_x, train_y=train_y, mean_module=means.ConstantMean(), small_noise=False
    ),
    shuffle_dataset=False,
    separate_dataset=specific_model_dataset,
)

In [21]:
analysis_res = debugger.regress_against_all_dataset(50)

analysis_res.fig

In [22]:
debugger.regress_on_top_of_test(30)

In [24]:
from autorocks.optimizer.acqf import AcqfOptimizerCfg
from botorch.sampling import SobolQMCNormalSampler
import autorocks.optimizer.acqf as acqf_wrapper


debugger.regression_at_step(
    40,
    acqf_wrapper=acqf_wrapper.qUpperConfidenceBoundWrapper(
        beta=0.2,
        sampler=SobolQMCNormalSampler(num_samples=1024),
        optimizer_cfg=AcqfOptimizerCfg(dim=sys_param.dimensions),
    ),
)